In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import spacy

In [ ]:
train_file = "../UP-1.0/en_ewt-up-train.conllu"
dev_file = "../UP-1.0/en_ewt-up-dev.conllu"
test_file = "../UP-1.0/en_ewt-up-test.conllu"

In [ ]:
train_data = pd.DataFrame() #read data to dataframe here
dev_data = pd.DataFrame() 
# train_data.head()

In [ ]:
# do preprocessing with features here

In [ ]:
target_col = 'some_column_name'

X = train_data.loc[:, train_data.columns != target_col]
X_dev = dev_data.loc[:, dev_data.columns != target_col]

y = train_data.target_col
y_dev = dev_data.target_col

In [ ]:
model = LogisticRegression().fit(X, y)

In [ ]:
y_pred = model.predict(X_dev)

In [ ]:
print("METRICS")
print('precision', precision_score(y_dev, y_pred), sep='\t')
print('recall', recall_score(y_dev, y_pred), sep='\t')
print('f1', f1_score(y_dev, y_pred), sep='\t')